In [ ]:
# !unzip "/content/brain_tumor_dataset.zip"

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image
import os

In [2]:
SIZE = 224
PATH = r"D:/College/Capstone Project 2/data"

In [4]:
SIZE = 224

path = PATH + r"/brain_tumor_dataset"

yes_tumor_path = path + r"/yes"
no_tumor_path = path + r"/no"

yes_tumor = os.listdir(yes_tumor_path)
no_tumor = os.listdir(no_tumor_path)

len(os.listdir(no_tumor_path)), len(os.listdir(yes_tumor_path))

(1501, 1503)

In [5]:
# Resize yes_tumor images
for p in yes_tumor:
    path = yes_tumor_path+"/"+p
    # resize_save(path)
    try:
        im = Image.open(path)
        im = im.resize((SIZE, SIZE))
        im.save(path)
    except:
        print(path)
        break
# Resize no_tumor
for p in no_tumor:
    path = no_tumor_path+"/"+p
    # resize_save(path)
    try:
        im = Image.open(path)
        im = im.resize((SIZE, SIZE))
        im.save(path)
    except:
        print(path)
        break

In [6]:
len(os.listdir(yes_tumor_path)), len(os.listdir(no_tumor_path))

(1503, 1501)

In [8]:
data = []
label = []

for p in yes_tumor:
  path = yes_tumor_path+"/"+p
  image = cv.imread(path)
  data.append(image)
  label.append(1)

for p in no_tumor:
  path = no_tumor_path+"/"+p
  image = cv.imread(path)
  data.append(image)
  label.append(0)

len(data), len(label) 
# data[0]

(3004, 3004)

In [9]:
data = np.array(data)/255.0
label = np.array(label)

data.shape, label.shape

((3004, 224, 224, 3), (3004,))

In [10]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(data, label, test_size = 0.20)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2403, 224, 224, 3), (601, 224, 224, 3), (2403,), (601,))

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [12]:
# Initializing CNN
cnn=tf.keras.models.Sequential()

# adding first layer
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[224,224,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

# adding second layer
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

# adding third layer
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

# Flattening
cnn.add(tf.keras.layers.Flatten())

# Fully connected layer
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

# Output layers
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

cnn.compile(optimizer=Adam(), loss = BinaryCrossentropy(),  metrics = ["accuracy"])
history = cnn.fit(x_train, y_train, epochs = 10, validation_data=(x_test, y_test), batch_size = 16)

cnn.save("Big_Brain_tumor_cnn_2.h5")

Epoch 1/10
151/151 [==============================] - 16s 28ms/step - loss: 0.5123 - accuracy: 0.7903 - val_loss: 0.3270 - val_accuracy: 0.8852
Epoch 2/10
151/151 [==============================] - 4s 25ms/step - loss: 0.2158 - accuracy: 0.9189 - val_loss: 0.1471 - val_accuracy: 0.9484
Epoch 3/10
151/151 [==============================] - 4s 25ms/step - loss: 0.1062 - accuracy: 0.9680 - val_loss: 0.1035 - val_accuracy: 0.9551
Epoch 4/10
151/151 [==============================] - 4s 25ms/step - loss: 0.0447 - accuracy: 0.9900 - val_loss: 0.0911 - val_accuracy: 0.9784
Epoch 5/10
151/151 [==============================] - 4s 25ms/step - loss: 0.0276 - accuracy: 0.9917 - val_loss: 0.1013 - val_accuracy: 0.9700
Epoch 6/10
151/151 [==============================] - 4s 25ms/step - loss: 0.0247 - accuracy: 0.9921 - val_loss: 0.0439 - val_accuracy: 0.9884
Epoch 7/10
151/151 [==============================] - 4s 25ms/step - loss: 0.0423 - accuracy: 0.9859 - val_loss: 0.0615 - val_accuracy: 0.983

In [14]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 256)       147712    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 256)      0

In [13]:
from sklearn.metrics import confusion_matrix, classification_report

In [15]:
y_preds = (cnn.predict(x_test) > 0.5).astype("int32")
y_preds = y_preds.squeeze()
(confusion_matrix(y_test, y_preds))


array([[310,   3],
       [  2, 286]])

In [16]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       313
           1       0.99      0.99      0.99       288

    accuracy                           0.99       601
   macro avg       0.99      0.99      0.99       601
weighted avg       0.99      0.99      0.99       601

